In [1]:
#Below are requirement packages! You can install them in you terminal or insider jupyter!
#必备 python 包，如果没有安装可以在命令行或者jupyter 内安装

!pip install astropy
!pip install skymapper
!pip install healpy
!pip install --pre astroquery
!pip install hanziconv
!pip install pandas

In [15]:
# import python libararies  导入必备的软件包

from astropy import *
from astroquery.vizier import Vizier
from astropy.coordinates import Angle
from astropy.table import Table,Column
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from matplotlib.pyplot import figure
from astropy.wcs import WCS
from astropy.io import fits
import pandas
import skymapper as skm
from hanziconv import HanziConv
%matplotlib auto
from itertools import groupby
import difflib

Using matplotlib backend: Qt5Agg


In [3]:
# Read and print consterllations in Chinese and English
# 读取并打印星座中文名和缩写
t_cn = pandas.read_fwf('consterllation_cn',widths=[4,30,10],header=None)

for i in range(len(t_cn)):
    print('%5s'%HanziConv.toSimplified(t_cn[2][i]),'    ',t_cn[0][i])

  仙女座      And
  唧筒座      Ant
  天燕座      Aps
  天鹰座      Aql
  宝瓶座      Aqr
  天坛座      Ara
  白羊座      Ari
  御夫座      Aur
  牧夫座      Boo
  大犬座      CMa
  小犬座      CMi
  猎犬座      CVn
  雕具座      Cae
  鹿豹座      Cam
  摩羯座      Cap
  船底座      Car
  仙后座      Cas
 半人马座      Cen
  仙王座      Cep
  鲸鱼座      Cet
  蝘蜓座      Cha
  圆规座      Cir
  巨蟹座      Cnc
  天鸽座      Col
  后发座      Com
  南冕座      CrA
  北冕座      CrB
  巨爵座      Crt
 南十字座      Cru
  乌鸦座      Crv
  天鹅座      Cyg
  海豚座      Del
  剑鱼座      Dor
  天龙座      Dra
  小马座      Equ
  波江座      Eri
  天炉座      For
  双子座      Gem
  天鹤座      Gru
  武仙座      Her
  时钟座      Hor
  长蛇座      Hya
  水蛇座      Hyi
 印第安座      Ind
  小狮座      LMi
  蝎虎座      Lac
  狮子座      Leo
  天兔座      Lep
  天秤座      Lib
  豺狼座      Lup
  天猫座      Lyn
  天琴座      Lyr
  山案座      Men
 显微镜座      Mic
  麒麟座      Mon
  苍蝇座      Mus
  矩尺座      Nor
  南极座      Oct
  蛇夫座      Oph
  猎户座      Ori
  孔雀座      Pav
  飞马座      Peg
  英仙座      Per
  凤凰座      Phe
  绘架座      Pic
  南鱼座      PsA
  双鱼座     

In [18]:
#  Manually input consterllation name here. 在此处输入星座缩写
constname = 'Aqr'
constnameX = constname
if constname == 'Tau':
    constnameX =' Tau'

index = t_cn.index[t_cn[0]==constname]
constfullname = t_cn[1][index[0]]
constfullnameCN = HanziConv.toSimplified(t_cn[2][index[0]])
print("中文名：",constfullnameCN,'，   英文名：',constfullname,'，   缩写：',constname)

中文名： 宝瓶座 ，   英文名： Aquarius ，   缩写： Aqr


In [19]:
# Load star charts and some pre-process 读取星表并预处理

catalog_list = Vizier.find_catalogs('Bright Star Catalogue, 5th')
print({k:v.description for k,v in catalog_list.items()})

Vizier.ROW_LIMIT = -1
catalogs = Vizier.get_catalogs(catalog_list.keys())

t = catalogs[0]
t_hyg = Table.read('hygdata_v3.txt',format='pandas.csv')
t_lines = Table.read('asterisms.csv')
t_hyg['ra']=t_hyg['ra']*15

catalog_list = Vizier.find_catalogs('The NGC 2000.0 Catalogue')
print({k: v.description for k, v in catalog_list.items()})

Vizier.ROW_LIMIT = -1
catalogs = Vizier.get_catalogs(catalog_list.keys())

t_NGC = catalogs[0]

t_NGC_NAME = catalogs[1]

t_NGC_const = t_NGC[t_NGC['Const'] == constname]

ra_NGC = []
dec_NGC = []
name_NGC = []
ra_Speical = []
dec_Speical = []
name_Speical = []
for i in range(len(t_NGC_const)):
    if t_NGC_const[i]['Name'] in t_NGC_NAME['Name']:
        ra_Speical.append(
            Angle(t_NGC_const[i]['RAB2000'], unit='hourangle').deg)
        dec_Speical.append(Angle(t_NGC_const[i]['DEB2000'], unit=u.deg).deg)
        name_Speical.append(t_NGC_NAME[t_NGC_NAME['Name'] == t_NGC_const[i]
                                       ['Name']]['Object'][0])
    else:
        ra_NGC.append(Angle(t_NGC_const[i]['RAB2000'], unit='hourangle').deg)
        dec_NGC.append(Angle(t_NGC_const[i]['DEB2000'], unit=u.deg).deg)
        if t_NGC_const[i]['Name'][0] == 'I':
            name_NGC.append('IC ' + t_NGC_const[i]['Name'][1::])
        else:
            name_NGC.append('NGC ' + t_NGC_const[i]['Name'])


for i in enumerate(name_Speical):
    if i[1][0]=='M' and i[1][1]==' ':
        str1 = ' '.join(i[1].split())
        name_Speical[i[0]]=str1.split(' ')[0]+str1.split(' ')[1]

print(name_Speical)

# Delete some unusful lines
rax = []
dex = []
delx = []
for i in range(len(t)):
    try:
        rax.append(Angle(t[i]['RAJ2000'],unit='hourangle').deg)
        dex.append(Angle(t[i]['DEJ2000'],unit=u.deg).deg)
    except:
        delx.append(i)

print(delx)
for x in delx[::-1]:
    del(t[x])
    
t_ra = Column(rax,name='ra_deg')
t_de = Column(dex,name='dec_deg')
t.add_column(t_ra,index=5)
t.add_column(t_de,index=6)
const = []
for i in range(len(t)):
   if constnameX in t['Name'][i]: 
        const.append(i)
t_const = np.copy(t[const])

{'V/50': 'Bright Star Catalogue, 5th Revised Ed. (Hoffleit+, 1991)'}
{'VII/118': 'NGC 2000.0 (Sky Publishing, ed. Sinnott 1988)', 'VII/155': 'Third Reference Cat. of Bright Galaxies (RC3) (de Vaucouleurs+ 1991)'}
['M72', 'M73', 'Saturn nebula', "Baxendell's unphotographable nebula", 'M2', 'Helix nebula']
[91, 94, 181, 1056, 1840, 2471, 2495, 3514, 3670, 6308, 6515, 7036, 7189, 8295]


In [20]:
# Set colors for bright stars 给亮星上色
color = []
for i in enumerate(t_const['SpType']):
    c =i[1].decode('UTF-8')[0]
    if   c == 'O': color.append('blue') ,
    elif c == 'B': color.append('deepskyblue'),
    elif c == 'A': color.append('cyan'),
    elif c == 'F': color.append('lemonchiffon'),
    elif c == 'G': color.append('lightyellow'),
    elif c == 'K': color.append('yellow'),
    elif c == 'M': color.append('orange'),
    else         : color.append('tomato'),
print(len(t_const))

91


In [21]:
#Find more stars in Hyg catalog 找到更多的Hyg星表中的恒星

ra  = t_const['ra_deg']
dec = t_const['dec_deg']
others = []

ra_max = np.max(ra)
ra_min = np.min(ra)
ra_mid = (ra_max+ra_min)/2

if ra_max-ra_min>200:
    ra_min=np.max(ra[ra<100])
    ra_max=np.min(ra[ra>100])
    #print(ra_min,ra_max)
    ra_min2 = ra_max
    ra_max  = ra_min + 360
    ra_min = ra_min2
    ra_mid = (ra_max+ra_min)/2

    if ra_mid>360:
        ra_mid = ra_mid-360
    for i in range(len(t_hyg)):
        if (ra_min2<t_hyg['ra'][i]<=360) and \
        (np.min(dec)<t_hyg['dec'][i]<=np.max(dec)):
            others.append(i)   
        if (0<t_hyg['ra'][i]<=ra_max-360) and \
        (np.min(dec)<t_hyg['dec'][i]<=np.max(dec)):
            others.append(i)  
else:
    for i in range(len(t_hyg)):
        if (ra_min<t_hyg['ra'][i]<=ra_max) and \
        (np.min(dec)<t_hyg['dec'][i]<=np.max(dec)):
            others.append(i)

ra_others=   t_hyg['ra'][others]
dec_others = t_hyg['dec'][others]
t_others = t_hyg[others]

In [22]:
# Translate name into Chinese 翻译中文

t_enggreece = Table.read('eng_greece.txt',format='pandas.csv')
#t_starCN2 = Table.read('starnameCN.csv') # Alternative Chinese star chart
t_starCN = Table.read('star_constCN.csv')
t_starCN_const = t_starCN[t_starCN['col2']==constname]

star_eng =t_starCN_const['col1']
gre_name = []
cn_name = []
star_eng = np.unique(star_eng)

ra_num=[]
dec_num=[]
name_num=[]

for i in star_eng:
    #print(i,t_starCN_const[t_starCN_const['col1']==i][0]['col3'])
    try:
        j =  t_const['Name']==str.encode(i.strip()+'    '+constname)
        j2 = np.where(j)[0][0]
        ra_num.append(t[const]['ra_deg'][j2])
        dec_num.append(t[const]['dec_deg'][j2])
        name_num.append(t_starCN_const[t_starCN_const['col1']==i][0]['col3'])
    except:
        pass
    try:
        s = t_enggreece[t_enggreece['en']==i[0]]['engname']
        cnname = t_starCN_const[t_starCN_const['col1']==i][0]['col3']
        gre_name.append(s[0])
        cn_name.append(cnname)
        print(constfullname+'%4s' %i,':','%10s' %s[0],'---->',cnname)
    except:
        pass

Aquarius   a :      alpha ----> 危宿一
Aquarius   b :       beta ----> 虚宿一
Aquarius   d :      delta ----> 羽林军二十六
Aquarius   e :    epsilon ----> 女 宿 一
Aquarius   g :      gamma ----> 坟墓二
Aquarius   k :      kappa ----> 虚梁三
Aquarius   q :      theta ----> 泣二
Aquarius   x :         xi ----> 天垒城一


In [23]:
# Load Greece letters 读取希腊字母
greek_alphabet = {
    u'\u0391': 'Alpha',
    u'\u0392': 'Beta',
    u'\u0393': 'Gamma',
    u'\u0394': 'Delta',
    u'\u0395': 'Epsilon',
    u'\u0396': 'Zeta',
    u'\u0397': 'Eta',
    u'\u0398': 'Theta',
    u'\u0399': 'Iota',
    u'\u039A': 'Kappa',
    u'\u039B': 'Lamda',
    u'\u039C': 'Mu',
    u'\u039D': 'Nu',
    u'\u039E': 'Xi',
    u'\u039F': 'Omicron',
    u'\u03A0': 'Pi',
    u'\u03A1': 'Rho',
    u'\u03A3': 'Sigma',
    u'\u03A4': 'Tau',
    u'\u03A5': 'Upsilon',
    u'\u03A6': 'Phi',
    u'\u03A7': 'Chi',
    u'\u03A8': 'Psi',
    u'\u03A9': 'Omega',
    u'\u03B1': 'alpha',
    u'\u03B2': 'beta',
    u'\u03B3': 'gamma',
    u'\u03B4': 'delta',
    u'\u03B5': 'epsilon',
    u'\u03B6': 'zeta',
    u'\u03B7': 'eta',
    u'\u03B8': 'theta',
    u'\u03B9': 'iota',
    u'\u03BA': 'kappa',
    u'\u03BB': 'lamda',
    u'\u03BC': 'mu',
    u'\u03BD': 'nu',
    u'\u03BE': 'xi',
    u'\u03BF': 'omicron',
    u'\u03C0': 'pi',
    u'\u03C1': 'rho',
    u'\u03C3': 'sigma',
    u'\u03C4': 'tau',
    u'\u03C5': 'upsilon',
    u'\u03C6': 'phi',
    u'\u03C7': 'chi',
    u'\u03C8': 'psi',
    u'\u03C9': 'omega',
}
greece_reverse = dict(map(reversed, greek_alphabet.items()))

greece_list = list(greek_alphabet.values())

In [24]:
#Pre-process for lines between stars. 星座连线预处理

t_line_const = t_lines[t_lines['constellation']==constname]
ra_line = t_line_const['ra'][0].split('[')[1].split(']')[0].split(',')
dec_line = t_line_const['dec'][0].split('[')[1].split(']')[0].split(',')
npair = int(t_line_const['num_pairs'])
plt_ra = [[float(ra_line[2*i])*15,
                 float(ra_line[2*i+1])*15] for i in range(npair)]
plt_dec = [[float(dec_line[2*i]),
            float(dec_line[2*i+1])] for i in range(npair)]

In [25]:
#Setting Plot area and title 设定绘画范围和标题

crit = skm.stdDistortion
proj = skm.Albers.optimize(ra,dec,crit=crit)

print(proj.lon_0,ra_mid)
proj.lon_0=ra_mid
title_ra = proj.lon_0
title_dec1 = (proj.lat_1+(proj.lat_2-proj.lat_1)*1.0)
title_dec2 = (proj.lat_1+(proj.lat_2-proj.lat_1)*0.9)
title_string1 =constfullnameCN
title_string2 ='知乎@凌晨晓骥'

optimizing parameters of Albers to minimize stdDistortion
best objective 0.006721 at Albers(340.36731790681785,-9.610833333333334,-19.306146257828345,-2.6564301982579286)
340.36731790681785 333.84624999999994


In [28]:
#Plot the graph!   画图 !

plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False

mapx = skm.Map(proj, facecolor='black')
sep = 5
mapx.grid(sep=sep, c='white', lw=0.5, alpha=0.3)
mapx.scatter(ra_others, dec_others, s=(17-t_others['mag'])**5/8e5, c='white')
mapx.scatter(ra,
             dec,
             s=(10 - t_const['Vmag'])**5 / 5e2,
             c=color,
             edgecolor=color,
             facecolor='white')
mapx.scatter(ra,
             dec,
             s=(10 - t_const['Vmag'])**5 / 6e2,
             alpha=0.9,
             c='white',
             edgecolor=color,
             facecolor='white')
mapx.focus(ra, dec)

dec_range = np.max(dec)-np.min(dec)

for i in enumerate(t_const['Name']):
    namex = i[1].decode('UTF-8').split(constname)[0]
    ss = [''.join(g) for _, g in groupby(namex, str.isalpha)]
    res = [i for i in ss if i.isalpha()]
    #print(res)
    if len(res):
        namey = res[0].lower()
        #print(namey)
        diflib = difflib.get_close_matches(namey, greece_list)
        for j in diflib:
            if j[0:2]==namey[0:2]:
                namez = j
        difflib.get_close_matches(namey, greece_list)[0]
        namezx = greece_reverse[namez]
        #print(namex,namey,namez,namezx)
        cn_add = ''
        if len(ss) > 2:
            if ss[2].strip() != '':
                cn_add= ss[2].strip()
                namezx = r'$' + namezx.strip() + '^{' + ss[2].strip() + '}$'
        else:
            namezx = r'$' +namezx.strip() + '$'
        if namez in gre_name:
            gre_index = gre_name.index(namez)
            mapx.text(ra[i[0]],
                  dec[i[0]] - dec_range/50,
                  cn_name[gre_index]+cn_add,
                  c='yellow',
                  fontsize=10,
                  ha='center',
                  va='center')    
        
        mapx.text(ra[i[0]],
                  dec[i[0]] + dec_range/50,
                  namezx,
                  c='lime',
                  fontsize=15,
                  ha='center',
                  va='center')
   
for i in range(len(ra_num)):
    mapx.text(ra_num[i],
              dec_num[i] - dec_range/50,
              name_num[i],
              c='yellow',
              fontsize=10,
              ha='center',
              va='center')
for i in range(npair):
    mapx.plot(np.asarray(plt_ra[i]),
              np.asarray(plt_dec[i]),
              color='white',
              alpha=0.5,
              lw=0.5)
if len(ra_Speical):
    mapx.scatter(np.asarray(ra_Speical),np.asarray(dec_Speical),
                 marker='o',c='',edgecolor='cyan',s=40,lw=0.5)
for i in range(len(ra_Speical)):
    print(i,name_Speical[i])
    mapx.text(ra_Speical[i],
              dec_Speical[i] + dec_range/100.0,
              name_Speical[i]
              ,fontsize=10,c='cyan',ha='center',va='center')
if len(ra_NGC):
    mapx.scatter(np.asarray(ra_NGC),np.asarray(dec_NGC),
                 marker='o',c='',edgecolor='magenta',s=5,lw=0.5,alpha=0.5)
# for i in range(len(ra_NGC)):
#     #print(i,name_NGC[i])
#     mapx.text(ra_NGC[i],
#               dec_NGC[i] + dec_range/100.0,
#               name_NGC[i]
#               ,fontsize=10,c='magenta',ha='center',va='center')
mapx.text(title_ra,title_dec1,title_string1,
          c='white',fontsize=30,alpha = 0.5,ha='center',va='center')
mapx.text(title_ra,title_dec2,title_string2,
          c='white',fontsize=20,alpha = 0.4,ha='center',va='center')

/home/jl/anaconda3/envs/astroconda/lib/python3.6/site-packages/matplotlib/collections.py:857: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
/home/jl/anaconda3/envs/astroconda/lib/python3.6/site-packages/skymapper/map.py:688: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  self.fig.tight_layout(pad=0.75)


0 M72
1 M73
2 Saturn nebula
3 Baxendell's unphotographable nebula
4 M2
5 Helix nebula


/home/jl/anaconda3/envs/astroconda/lib/python3.6/site-packages/skymapper/map.py:810: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  self.fig.tight_layout(pad=0.75)


Text(-0.0, 0.09303007914710015, '知乎@凌晨晓骥')